## 0. 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

products = pd.read_csv("../dataset/products.csv")
users = pd.read_csv("../dataset/users.csv")
ratings = pd.read_csv("../dataset/ratings.csv")

In [ ]:
print(f"products: {products.shape}")
display(products.head())

print(f"users: {users.shape}")
display(users.head())

print(f"ratings: {ratings.shape}")
display(ratings.head())

## 1. 내용 기반 추천 (상품 설명 + 태그 + 카테고리 활용)
- 속성 기반 매칭 → 동일 속성을 공유하는 아이템을 우선 추천(카테고리/태그 기반 필터링)
- 머신러닝 기반 → 아이템의 속성을 독립 변수로, 사용자가 좋아할 확률을 추정
- 속성 기반 유사도 → 아이템 관련 속성들 임베딩 후 유사도 측정 방법론 사용

### 1) 데이터 전처리

In [ ]:
# 텍스트 특성 결합
# 휴대용, 터치스크린, 최신 기술과 소음 제거 기능 → 휴대용 터치스크린 최신 기술과 소음 제거 기능

products["features"] = products["description"] + " " + products["tags"].str.replace(",", "") + " " + products["category"]

products.head()

### 2) 데이터 벡터화

#### [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)

- Bag of Words(단어들의 출현 빈도에만 집중하는 텍스트 데이터 수치화 표현 방법) 기법을 활용한 벡터화 방식
- 문서 내에서 단어의 등장 횟수를 기반으로 벡터를 생성
- 빈도만을 고려하기 때문에 모든 단어가 같은 중요도를 가짐
- 예시
  - 문서: `"yesterday all my trouble seem so far away"`
  - 단어 사전: `[yesterday, all, my, trouble, seem, far]`
  - 벡터: `[1, 1, 1, 1, 1, 1]`(각 단어가 한 번씩 등장했으므로 모두 1)

---

#### [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

- TF-IDF(Term Frequency-Inverse Document Frequency) 기법을 기반으로 벡터 변환을 수행
- 단어의 빈도와 역 문서 빈도를 사용하여 단어들마다 중요한 정도에 따라서 가중치를 부여하는 방법
- TF (Term Frequency, 단어 빈도) : 한 문서 안에서 어떤 단어가 얼마나 자주 나오는지를 보는 값
- IDF (Inverse Document Frequency, 역문서 빈도) : 흔한 단어일수록 덜 중요하게, 드문 단어일수록 더 중요하게 보정하는 값
- 문서에 많이 등장하는 단어는 중요하다고 가정하지만, 모든 문서에 걸쳐 자주 등장하는 단어(the, is, at, this 등)는 정보량이 적기 때문에 가중치가 낮아진다.
- 예시
  - 문서: `"yesterday all my trouble seem so far away"`
  - 단어 사전: `[yesterday, all, my, trouble, seem, far]`
  - 벡터: `[0.52, 0.30, 0.30, 0.45, 0.45, 0.40]`(`yesterday`, `trouble` 같은 특정성이 높은 단어에 더 큰 가중치가 부여됨)

In [ ]:
# TfidfVectorizer 적용

tfidf = TfidfVectorizer(
    ngram_range=(1, 3),
)

tfidf_matrix = tfidf.fit_transform(products["features"])

tfidf_matrix.shape

### 3) 유사도 계산 - 코사인 유사도

| user_id |  |  |  |  |
|----------|-----|-----|-----|-----|
| 1        | 5.0 | 4.0 | 3.0 | 0.0 |
| 2        | 4.0 | 0.0 | 0.0 | 5.0 |
| 3        | 0.0 | 2.0 | 3.0 | 0.0 |
| 4        | 5.0 | 0.0 | 4.0 | 2.0 |
---
#### Step 1: 두 사용자의 평점을 벡터로 표현
사용자 1과 사용자 2의 평점 벡터를 추출합니다.

- 사용자 1의 벡터 : A = [5.0, 4.0, 3.0, 0.0]
- 사용자 2의 벡터 : B = [4.0, 0.0, 0.0, 5.0]
---
#### Step 2: 벡터 내적 계산
각 원소 간의 곱을 더한다.
- `A⋅B=(5.0×4.0)+(4.0×0.0)+(3.0×0.0)+(0.0×5.0)=20.0+0.0+0.0+0.0=20.0`
---
#### Step 3: 벡터의 크기(유클리드 거리) 계산

- A_Euclid : `np.sqrt(25 + 14 + 9)`
- B_Euclid : `np.sqrt(16 + 25)`
---
#### Step 4: 코사인 유사도 계산

- Cosine Similarity = `A⋅B / (A_Euclid + B_Euclid)`
---
#### Step 5: 나머지 사용자들에 대해서도 반복
- user1 - user1 -> 1
- user1 - user2 -> 0.442
- user1 - user3 -> 0.667
- user1 - user4 -> 0.779
- 최종적으로 user1에 대한 코사인 유사도 결과 행 생성 -> [1, 0.442, 0.667, 0.779]

In [ ]:
# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print(cosine_sim.shape)

cosine_sim[:5, :5]

In [ ]:
# 1) 유사도 점수 정렬

prod_id = 1

sim_scores = list(enumerate(cosine_sim[prod_id-1]))
sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

print(sim_scores)

In [ ]:
# 2) 연관 상품 인덱스 출력 (자기 자신 제외)

sim_scores = sim_scores[1:]
product_indices = [i[0] for i in sim_scores]

print(product_indices)

### 4) 추천 함수 생성

In [ ]:
def contents_recommendation(dataframe, sim_matrix, product_id, top_k=7):

    # 유사도 점수 정렬
    sim_scores = list(enumerate(sim_matrix[product_id-1]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 상위 top_k 추천 (자기 자신 제외)
    sim_scores = sim_scores[1:top_k+1]
    product_indices = [i[0] for i in sim_scores]

    print(f"기준 상품 : ", dataframe.loc[product_id-1]["name"])

    return dataframe.iloc[product_indices][["product_id", "name", "category", "brand", "description"]]

print("🔹 콘텐츠 기반 추천")
contents_recommendation(
    dataframe=products,
    sim_matrix=cosine_sim,
    product_id=1,
    top_k=7
    )

## 2. 사용자 기반 협업 필터링
- 사용자들이 아이템들을 어떻게 평가했는지를 분석하여 비슷하게 분석한 사용자들이 선호하는 아이템을 추천

### 1) USER-ITEM 행렬 생성

In [ ]:
user_item_matrix = ratings.pivot_table(
    index="user_id",
    columns="product_id",
    values="rating",
    aggfunc="mean",
    fill_value=0
    )
    
user_item_matrix

### 2) 유사도 계산

In [ ]:
# 코사인 유사도(사용자-사용자)
user_sim = cosine_similarity(user_item_matrix)

user_sim[:5, :5]

In [ ]:
target_user = 13

# 유사도 행렬에서는 인덱스가 0부터 시작하기 때문에 사용자 번호에서 1을 뺍니다.
sim_scores = list(enumerate(user_sim[target_user-1]))
sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)

# 자기 자신 제외
sim_scores = sim_scores[1:]

In [ ]:
# 사용자가 구매한 아이템 ID 추출
purchased_item = [i[0]+1 for i in list(enumerate(user_item_matrix.loc[target_user])) if i[1] != 0]

# 유사도가 높은 아이템 ID 추출
product_indices = [i[0]+1 for i in sim_scores]

In [ ]:
# 사용자가 구매하지 않은 추천 아이템 ID 추출
new_item = [i for i in product_indices if i not in purchased_item]

print(new_item)

In [ ]:
# 상품 테이블에서 해당 상품 정보 출력

products.set_index('product_id').loc[new_item].reset_index()


### 3) 추천 함수 생성

In [ ]:
# 사용자 기반 협업 필터링 추천 함수 생성

def user_col_filtering(dataframe, sim_matrix, user_id, top_k=7):

    sim_scores = list(enumerate(sim_matrix[user_id-1]))
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    
    # 상위 top_k 추천 (자기 자신 제외)
    sim_scores = sim_scores[1:top_k+1]

    purchased_item = [i[0]+1 for i in list(enumerate(user_item_matrix.loc[user_id])) if i[1] != 0]

    product_indices = [i[0]+1 for i in sim_scores]        

    # 사용자가 구매하지 않은 추천 아이템 ID 추출
    new_item = [i for i in product_indices if i not in purchased_item]
    
    # 추천 상품이 모두 사용자가 구매한 상품인 경우에는 추천 아이템 중 상위 3개 출력
    if not new_item:
        new_item = product_indices[:3]
    
    user_id_info = tuple(users[users["user_id"] == user_id].iloc[0].tolist())
    print(f"기준 유저 : {user_id_info}")

    # new_item 순서대로 물품 디테일 return
    return dataframe.set_index('product_id').loc[new_item].reset_index()

user_col_filtering(
    dataframe=products,
    sim_matrix=user_sim,
    user_id=12,
    top_k=7
)

## 3. 아이템 기반 협업 필터링
- 사용자들이 아이템들을 어떻게 평가했는지를 분석하여 비슷한 흐름으로 평가받은 아이템을 추천

### 1) 유사도 계산

In [ ]:
# 아이템을 기준으로 계산하기 위해 행렬을 전치하여 코사인 유사도 함수에 적용

item_sim = cosine_similarity(user_item_matrix.T)

item_sim[:5, :5]

In [ ]:
# 특정 아이템을 검색했을 때 그와 연관된 아이템들이 추천되도록!

target_item = 8

# 유사도 행렬에서는 인덱스가 0부터 시작하기 때문에 사용자 번호에서 1을 뺍니다.
sim_scores = list(enumerate(item_sim[target_item-1]))
sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)

# 자기 자신 제외
sim_scores = sim_scores[1:]

# 유사도가 높은 아이템 ID 추출
product_indices = [i[0]+1 for i in sim_scores]

product_indices

In [ ]:
# 상품 테이블에서 TOP5 상품 정보 출력

products[products['product_id'].isin(product_indices)][:5]

### 2) 추천 함수 생성

In [ ]:
def item_col_filtering(dataframe, sim_matrix, product_id, top_k=7):
    
    sim_scores = list(enumerate(sim_matrix[product_id-1]))
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse=True)
    
    # 상위 top_k 추천 (자기 자신 제외)
    sim_scores = sim_scores[1:top_k+1]
    
    product_indices = [i[0]+1 for i in sim_scores]
    
    print(f"기준 상품 : {dataframe[dataframe['product_id'] == product_id]['name'].values[0]}")

    return dataframe.set_index('product_id').loc[product_indices].reset_index()[:5]

item_col_filtering(
    dataframe=products,
    sim_matrix=item_sim,
    product_id=12,
    top_k=7
)

## 4. 하이브리드 추천 (콘텐츠 기반 + 협업 필터링 결합)
- 내용 기반 추천 + 유저 기반 협업필터링 + 아이템 기반 협업필터링
- 세 가지 방법들에 각각 일정 가중치를 부여하여 최종 상품을 추천

In [ ]:
target_item = 1
target_user = 10

content_rec = contents_recommendation(
    dataframe=products,
    sim_matrix=cosine_sim,
    product_id=target_item,
    )

user_rec = user_col_filtering(
    dataframe=products,
    sim_matrix=user_sim,
    user_id=target_user,
    )

item_rec = item_col_filtering(
    dataframe=products,
    sim_matrix=item_sim,
    product_id=target_item,
    )

In [ ]:
from collections import Counter

def hybrid_recommend(dataframe, rec1, rec2, rec3, top_k=5, **alpha):
    
    weight1 = alpha.get('weight1')
    weight2 = alpha.get('weight2')
    weight3 = alpha.get('weight3')
    
    # 1. 추천된 상품들을 하나의 리스트로 합치기
    all_products = []
    
    all_products = rec1['product_id'].tolist()*weight1 \
        + rec2['product_id'].tolist()*weight2 \
        + rec3['product_id'].tolist()*weight3
        
    # 2. 상품별 등장 횟수 계산 (가중치 합산)
    product_counts = Counter(all_products)
    
    # 3. 등장 횟수 순으로 정렬하여 상위 top_k개 선택
    top_products = [prod_id for prod_id, count in product_counts.most_common(top_k)]

    # 4. 상품 정보 반환
    result = dataframe[dataframe['product_id'].isin(top_products)].copy()
    result['recommendation_score'] = result['product_id'].map(product_counts)
    
    return result[['product_id', 'name', 'category', 'brand', 'recommendation_score']].sort_values('recommendation_score', ascending=False)

print("🔹 하이브리드 추천")
alpha = dict(weight1=3, weight2=1, weight3=2)

hybrid_recommend(
    dataframe = products,
    rec1 = content_rec,
    rec2 = user_rec,
    rec3 = item_rec,
    **alpha
    )